# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 18 2022 11:40AM,242074,22,621124,"NBTY Global, Inc.",Executing
1,Jul 18 2022 11:35AM,242020,10,7969500.1,"Citieffe, Inc.",Released
2,Jul 18 2022 11:35AM,242020,10,CTF0011223.1,"Citieffe, Inc.",Released
3,Jul 18 2022 11:35AM,242020,10,CTF0011248.1,"Citieffe, Inc.",Released
4,Jul 18 2022 11:35AM,242020,10,CTF0011249.1,"Citieffe, Inc.",Released
5,Jul 18 2022 11:35AM,242020,10,8041055.1,"Citieffe, Inc.",Released
6,Jul 18 2022 11:35AM,242020,10,CTF0011228.1,"Citieffe, Inc.",Released
7,Jul 18 2022 11:35AM,242020,10,CTF0011262.2,"Citieffe, Inc.",Released
8,Jul 18 2022 11:35AM,242020,10,CTF001125.1,"Citieffe, Inc.",Released
9,Jul 18 2022 11:35AM,242020,10,CTF0011270.1,"Citieffe, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,242069,Released,1
32,242070,Released,1
33,242071,Executing,1
34,242072,Released,1
35,242074,Executing,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
242069,NaN,1.0
242070,NaN,1.0
242071,1.0,NaN
242072,NaN,1.0
242074,1.0,NaN


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
241972,0.0,14.0
242005,0.0,3.0
242007,0.0,1.0
242009,0.0,1.0
242012,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
241972,0,14
242005,0,3
242007,0,1
242009,0,1
242012,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,241972,0,14
1,242005,0,3
2,242007,0,1
3,242009,0,1
4,242012,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,241972,,14
1,242005,,3
2,242007,,1
3,242009,,1
4,242012,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 18 2022 11:40AM,242074,22,"NBTY Global, Inc."
1,Jul 18 2022 11:35AM,242020,10,"Citieffe, Inc."
20,Jul 18 2022 11:32AM,242072,21,"NBTY Global, Inc."
21,Jul 18 2022 11:28AM,242071,21,"NBTY Global, Inc."
22,Jul 18 2022 11:24AM,242070,19,"GCH Granules USA, Inc."
23,Jul 18 2022 11:20AM,242069,21,"NBTY Global, Inc."
24,Jul 18 2022 11:10AM,242068,21,"NBTY Global, Inc."
25,Jul 18 2022 10:45AM,242066,19,"Innogenix, LLC"
50,Jul 18 2022 10:44AM,242067,20,"ACI Healthcare USA, Inc."
51,Jul 18 2022 10:32AM,241972,20,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 18 2022 11:40AM,242074,22,"NBTY Global, Inc.",1,
1,Jul 18 2022 11:35AM,242020,10,"Citieffe, Inc.",,19
2,Jul 18 2022 11:32AM,242072,21,"NBTY Global, Inc.",,1
3,Jul 18 2022 11:28AM,242071,21,"NBTY Global, Inc.",1,
4,Jul 18 2022 11:24AM,242070,19,"GCH Granules USA, Inc.",,1
5,Jul 18 2022 11:20AM,242069,21,"NBTY Global, Inc.",,1
6,Jul 18 2022 11:10AM,242068,21,"NBTY Global, Inc.",,1
7,Jul 18 2022 10:45AM,242066,19,"Innogenix, LLC",6,19
8,Jul 18 2022 10:44AM,242067,20,"ACI Healthcare USA, Inc.",,1
9,Jul 18 2022 10:32AM,241972,20,"ACI Healthcare USA, Inc.",,14


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 18 2022 11:40AM,242074,22,"NBTY Global, Inc.",,1
1,Jul 18 2022 11:35AM,242020,10,"Citieffe, Inc.",19,
2,Jul 18 2022 11:32AM,242072,21,"NBTY Global, Inc.",1,
3,Jul 18 2022 11:28AM,242071,21,"NBTY Global, Inc.",,1
4,Jul 18 2022 11:24AM,242070,19,"GCH Granules USA, Inc.",1,
5,Jul 18 2022 11:20AM,242069,21,"NBTY Global, Inc.",1,
6,Jul 18 2022 11:10AM,242068,21,"NBTY Global, Inc.",1,
7,Jul 18 2022 10:45AM,242066,19,"Innogenix, LLC",19,6
8,Jul 18 2022 10:44AM,242067,20,"ACI Healthcare USA, Inc.",1,
9,Jul 18 2022 10:32AM,241972,20,"ACI Healthcare USA, Inc.",14,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 18 2022 11:40AM,242074,22,"NBTY Global, Inc.",,1
1,Jul 18 2022 11:35AM,242020,10,"Citieffe, Inc.",19,
2,Jul 18 2022 11:32AM,242072,21,"NBTY Global, Inc.",1,
3,Jul 18 2022 11:28AM,242071,21,"NBTY Global, Inc.",,1
4,Jul 18 2022 11:24AM,242070,19,"GCH Granules USA, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 18 2022 11:40AM,242074,22,"NBTY Global, Inc.",NaN,1.0
1,Jul 18 2022 11:35AM,242020,10,"Citieffe, Inc.",19.0,NaN
2,Jul 18 2022 11:32AM,242072,21,"NBTY Global, Inc.",1.0,NaN
3,Jul 18 2022 11:28AM,242071,21,"NBTY Global, Inc.",NaN,1.0
4,Jul 18 2022 11:24AM,242070,19,"GCH Granules USA, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 18 2022 11:40AM,242074,22,"NBTY Global, Inc.",0.0,1.0
1,Jul 18 2022 11:35AM,242020,10,"Citieffe, Inc.",19.0,0.0
2,Jul 18 2022 11:32AM,242072,21,"NBTY Global, Inc.",1.0,0.0
3,Jul 18 2022 11:28AM,242071,21,"NBTY Global, Inc.",0.0,1.0
4,Jul 18 2022 11:24AM,242070,19,"GCH Granules USA, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2662254,185.0,65.0
12,242048,36.0,9.0
15,726184,21.0,10.0
16,242056,0.0,1.0
19,726160,20.0,7.0
20,1452173,42.0,0.0
21,1452407,4.0,2.0
22,242074,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2662254,185.0,65.0
1,12,242048,36.0,9.0
2,15,726184,21.0,10.0
3,16,242056,0.0,1.0
4,19,726160,20.0,7.0
5,20,1452173,42.0,0.0
6,21,1452407,4.0,2.0
7,22,242074,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,185.0,65.0
1,12,36.0,9.0
2,15,21.0,10.0
3,16,0.0,1.0
4,19,20.0,7.0
5,20,42.0,0.0
6,21,4.0,2.0
7,22,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,185.0
1,12,Released,36.0
2,15,Released,21.0
3,16,Released,0.0
4,19,Released,20.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Executing,65.0,9.0,10.0,1.0,7.0,0.0,2.0,1.0
Released,185.0,36.0,21.0,0.0,20.0,42.0,4.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Executing,65.0,9.0,10.0,1.0,7.0,0.0,2.0,1.0
1,Released,185.0,36.0,21.0,0.0,20.0,42.0,4.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Executing,65.0,9.0,10.0,1.0,7.0,0.0,2.0,1.0
1,Released,185.0,36.0,21.0,0.0,20.0,42.0,4.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()